In [2]:
# Image Data Generator 
# How to make folder structure for Image Data generator 
# DATA (FOLDER)
#    -> Category1 (FOLDER)
#          -> Image1.1 (FILE)
#          -> Image1.2
#          -> Image1.3
#    -> Category2  (FOLDER)
#          -> Image2.1 (FILE)
#          -> Image2.2
#          -> Image2.3

In [15]:
import os
import shutil
import pandas as pd

In [21]:
all_image_name = os.listdir("dogs-vs-cats/train/train")
df = pd.DataFrame(all_image_name, columns=["image_name"])
df.head()
df2 = df.image_name.str.split(".", expand=True)

In [27]:
classes = df2[0].unique().tolist()
classes

['cat', 'dog']

In [5]:
for _class in classes:
    os.mkdir("data/" + _class)

In [11]:
SRC_FOLDER = "dogs-vs-cats/train/train/"
CAT_FOLDER = "data/cat/"
DOG_FODLER = "data/dog/"

In [12]:
for image in os.listdir(SRC_FOLDER):
    if image.lower().startswith("cat"):
        shutil.copy(SRC_FOLDER + image, CAT_FOLDER + image)
    else:
        shutil.copy(SRC_FOLDER + image, DOG_FODLER + image)

In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [31]:
dataGenerator = ImageDataGenerator(rotation_range=40, horizontal_flip=True, zoom_range=0.2, rescale=1/255)


In [33]:
#Image Augumentation 
#This is just for illustration
img = load_img("data/cat/cat.10.jpg")
img = img_to_array(img)
img = img.reshape((1,) + img.shape)


In [35]:
#This is just for illustration
i=0
os.mkdir("test")
for batch  in dataGenerator.flow(img, batch_size=1, save_to_dir="test", save_format="jpg"):
    i+=1
    if i>20:
        break

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Flatten, Dropout, MaxPooling2D, Conv2D
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD

In [44]:
model = Sequential()
#chunk-1
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(150,150,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
#chunk-2  
model.add(Conv2D(32, kernel_size=(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
#chunk-3  
model.add(Conv2D(64, kernel_size=(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#flattenning
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2,activation="softmax"))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       

In [46]:
model.compile(loss=categorical_crossentropy, optimizer="rmsprop", metrics=["acc"])

In [47]:
dataGenerator = ImageDataGenerator(rotation_range=40, horizontal_flip=True, zoom_range=0.2, rescale=1/255)

In [48]:
train_generator = dataGenerator.flow_from_directory("data", target_size=(150,150), batch_size=64)

Found 25000 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator, epochs=10)

Epoch 1/10
391/391 [==============================] - 1136s 3s/step - loss: 0.6630 - acc: 0.6108
Epoch 2/10
 63/391 [===>..........................] - ETA: 17:39 - loss: 0.5834 - acc: 0.6922